# Chapter 1: PQk-means
This chapter contains the followings:

1. Vector compression by Product Quantization
1. Clustering by PQk-means
1. Comparison to other clustering methods

Requisites:
- numpy
- sklearn
- pqkmeans

## 1. Vector compression by Product Quantization

In [1]:
import numpy
import pqkmeans
import sys
import pickle

First , we introduce vector compression by Product Quantization (PQ) [Jegou, TPAMI 11]. The first task is to train an encoder. Let us assume that there are 1000 six-dimensional vectors for training; $X_1 \in \mathbb{R}^{1000\times6}$



In [2]:
X1 = numpy.random.random((1000, 6))
print("X1.shape =  \n", X1.shape, "\n")
print("X1 = \n", X1)

X1.shape =  
 (1000, 6) 

X1 = 
 [[ 0.41924871  0.39184232  0.25803356  0.65040351  0.21106254  0.72738996]
 [ 0.33881997  0.4123091   0.69142687  0.45186094  0.19894196  0.06499685]
 [ 0.47682624  0.43083143  0.38538358  0.1519837   0.60178826  0.54658913]
 ..., 
 [ 0.11577582  0.43086611  0.27831923  0.60215751  0.62279282  0.65356463]
 [ 0.20633001  0.09706879  0.26794791  0.0876635   0.42875445  0.50787089]
 [ 0.60334538  0.41343645  0.10305301  0.34764159  0.16666211  0.69682394]]


Then we can train a PQEncoder using $X_1$.

In [3]:
encoder = pqkmeans.encoder.PQEncoder(num_subdim=2, Ks=256)
encoder.fit(X1)

The encoder takes two parameters: $num\_subdim$ and $Ks$. In the training step, each vector is splitted into $num\_subdim$ sub-vectors, and quantized with $Ks$ codewords. The $num\_subdim$ decides the bit length of PQ-code, and typically set as 4, 8, etc. The $Ks$ is usually set as 256.

In this example, each 6D training vector is splitted into $num\_subdim(=2)$ sub-vectors (two 3D vectors). Consequently, the 1000 6D training vectors are splitted into the two set of 1000 3D vectors. The k-means clustering is applied for each set of subvectors with $K=256$.


After the training step, the encoder stores the resulting codewords (2 subpspaces $*$ 256 codewords $*$ 3 dimensions):

In [4]:
print(encoder.codewords.shape)

(2, 256, 3)


Note that you can train the encoder preliminary using training data, and write/read the encoder via pickle.


In [5]:
# pickle.dump(encoder, open('encoder.pkl', 'wb'))  # Write
# encoder = pickle.load(open('encoder.pkl', 'rb'))  # Read

Next, let us consider database vectors (2000 six-dimensional vectors, $X_2$) that we'd like to compress. 

In [6]:
X2 = numpy.random.random((2000, 6))
print("X2.shape:\n", X2.shape, "\n")
print("X2:\n", X2, "\n")
print("Data type of each element:\n", type(X2[0][0]), "\n")
print("Memory usage:\n", X2.nbytes, "byte")

X2.shape:
 (2000, 6) 

X2:
 [[ 0.69521066  0.473434    0.81014451  0.83199628  0.53207691  0.68057145]
 [ 0.92873026  0.79807992  0.4145905   0.32912862  0.01921358  0.30650001]
 [ 0.71014149  0.10309274  0.99991603  0.49397926  0.66469458  0.32774237]
 ..., 
 [ 0.29750077  0.53425779  0.62608256  0.0462519   0.99887679  0.85922414]
 [ 0.45633923  0.88007103  0.89685826  0.50982455  0.15277933  0.84326488]
 [ 0.55456954  0.556289    0.86725168  0.72495792  0.8088668   0.36371054]] 

Data type of each element:
 <class 'numpy.float64'> 

Memory usage:
 96000 byte


We can compress these vectors by the trained PQ-encoder.

In [7]:
X2_pqcode = encoder.transform(X2)
print("X2_pqcode.shape:\n", X2_pqcode.shape, "\n")
print("X2_pqcode\n", X2_pqcode, "\n")
print("Data type of each element:\n", type(X2_pqcode[0][0]), "\n")
print("Memory usage:\n", X2_pqcode.nbytes, "byte")

X2_pqcode.shape:
 (2000, 2) 

X2_pqcode
 [[ 91  10]
 [ 16 209]
 [125 225]
 ..., 
 [203  20]
 [114  97]
 [226 153]] 

Data type of each element:
 <class 'numpy.uint8'> 

Memory usage:
 4000 byte


Each vector is splitted into $num\_subdim(=2)$ sub-vectors, and the nearest codeword is searched for each sub-vector. The id of the nearest codeword is recorded, i.e., two integers in this case. This representation is called PQ-code.
 
Note that PQ-code is a mamemory efficient data representation. The original 6D vector requies $6 * 64 = 384$ bit if 64 bit float is used for each element. On the other, PQ code requires only $2 * \log_2 256 = 16$ bit. 

We can approximately recunstruct the original vector from a PQ-code, by fetching the codewords using the PQ-code:

In [8]:
X2_reconstructed = encoder.inverse_transform(X2_pqcode)
print("original X2:\n", X2, "\n")
print("reconstructed X2:\n", X2_reconstructed)

original X2:
 [[ 0.69521066  0.473434    0.81014451  0.83199628  0.53207691  0.68057145]
 [ 0.92873026  0.79807992  0.4145905   0.32912862  0.01921358  0.30650001]
 [ 0.71014149  0.10309274  0.99991603  0.49397926  0.66469458  0.32774237]
 ..., 
 [ 0.29750077  0.53425779  0.62608256  0.0462519   0.99887679  0.85922414]
 [ 0.45633923  0.88007103  0.89685826  0.50982455  0.15277933  0.84326488]
 [ 0.55456954  0.556289    0.86725168  0.72495792  0.8088668   0.36371054]] 

reconstructed X2:
 [[ 0.7305386   0.41790788  0.91171652  0.79356276  0.55726669  0.63313401]
 [ 0.93921649  0.84747755  0.49089569  0.32181604  0.06143173  0.2127218 ]
 [ 0.67338156  0.09242146  0.95811486  0.53955805  0.57155243  0.27144563]
 ..., 
 [ 0.34576532  0.5893726   0.53532704  0.07156711  0.94468507  0.86145908]
 [ 0.42363968  0.85768321  0.81125923  0.47217088  0.16601343  0.72239844]
 [ 0.47092851  0.58453229  0.88074799  0.77164221  0.82516589  0.37545323]]


As can be seen, the reconstructed vectors are similar to the original one.

In a large-scale data processing scenario where all data cannot be stored on memory, you can compress input vectors to PQ-codes and store the PQ-codes only (X2_pqcode).

In [9]:
# pickle.dump(X2_pqcode, open('pqcode.pkl', 'wb')) # You can store the PQ-codes only

## 2. Clustering by PQk-means

Let us run the clustering over the PQ-codes. The clustering object is instanciated with the trained encoder. Here, we set the number of cluster as $k=10$.

In [10]:
kmeans = pqkmeans.clustering.PQKMeans(encoder=encoder, k=10)

Let's run the PQk-means over X2_pqcode.

In [11]:
clustered = kmeans.fit_predict(X2_pqcode)
print(clustered[:100]) # Just show the 100 results

[5, 8, 0, 7, 9, 7, 6, 1, 6, 2, 4, 9, 4, 4, 7, 7, 2, 7, 5, 2, 7, 0, 6, 7, 6, 2, 6, 0, 8, 2, 1, 9, 9, 5, 8, 5, 9, 2, 6, 8, 7, 6, 6, 5, 1, 5, 4, 7, 7, 4, 4, 8, 9, 9, 7, 3, 2, 6, 0, 8, 8, 5, 9, 8, 2, 8, 9, 5, 8, 8, 6, 5, 1, 7, 3, 5, 8, 0, 0, 7, 3, 4, 9, 8, 4, 0, 2, 4, 1, 5, 2, 9, 8, 5, 4, 0, 1, 4, 5, 2]


The resulting vector (clustered) contains the id of assigned codeword for each input PQ-code.

In [12]:
print("The id of assigned codeword for the 1st PQ-code is ", clustered[0])
print("The id of assigned codeword for the 2nd PQ-code is ", clustered[1])
print("The id of assigned codeword for the 3rd PQ-code is ", clustered[2])

The id of assigned codeword for the 1st PQ-code is  5
The id of assigned codeword for the 2nd PQ-code is  8
The id of assigned codeword for the 3rd PQ-code is  0


You can fetch the center of the clustering by:

In [13]:
print("clustering centers:\n", kmeans.cluster_centers_)

clustering centers:
 [[51, 248], [249, 197], [107, 249], [154, 49], [14, 16], [65, 165], [55, 148], [146, 225], [211, 55], [175, 159]]


The centers are also PQ-codes. They can be reconstructed by PQ-encoder. 

In [14]:
clustering_centers_numpy = numpy.array(kmeans.cluster_centers_, dtype=encoder.code_dtype)  # Convert to np.array with the proper dtype
clustering_centers_reconstructd = encoder.inverse_transform(clustering_centers_numpy) # From PQ-code to 6D vectors
print("reconstructed clustering centers:\n", clustering_centers_reconstructd)

reconstructed clustering centers:
 [[ 0.34063962  0.46187204  0.94672412  0.63280575  0.44489968  0.229906  ]
 [ 0.59851806  0.59250557  0.23761559  0.37800664  0.65305798  0.39798818]
 [ 0.28276182  0.49910144  0.31672088  0.67482735  0.34874969  0.76095219]
 [ 0.44099195  0.221538    0.83921304  0.62238899  0.28546003  0.70339381]
 [ 0.86247083  0.1914981   0.3646877   0.57631009  0.2564538   0.28428649]
 [ 0.63263658  0.66467431  0.69727081  0.64420818  0.70553312  0.76480393]
 [ 0.24648559  0.77371957  0.40618459  0.44454695  0.49833222  0.4897001 ]
 [ 0.39258137  0.28525802  0.41045738  0.53955805  0.57155243  0.27144563]
 [ 0.73492665  0.64110944  0.52159744  0.34383123  0.26950374  0.52341666]
 [ 0.50715649  0.33016919  0.63211549  0.15936672  0.64721403  0.75495636]]


Let's summalize the result:

In [15]:
print("13th input vector:\n", X2[12], "\n")
print("13th PQ code:\n", X2_pqcode[12], "\n")
print("reconstructed 13th PQ code:\n", X2_reconstructed[12], "\n")
print("ID of the assigned center:\n", clustered[12], "\n")
print("Assigned center (PQ-code):\n", kmeans.cluster_centers_[clustered[12]], "\n")
print("Assigned center (reconstructed):\n", clustering_centers_reconstructd[clustered[12]])

13th input vector:
 [ 0.87634215  0.2022789   0.24760898  0.41616456  0.37486582  0.33386463] 

13th PQ code:
 [ 20 203] 

reconstructed 13th PQ code:
 [ 0.96218507  0.2206644   0.3197819   0.44805649  0.37104167  0.39023411] 

ID of the assigned center:
 4 

Assigned center (PQ-code):
 [14, 16] 

Assigned center (reconstructed):
 [ 0.86247083  0.1914981   0.3646877   0.57631009  0.2564538   0.28428649]


## 3. Comparison to other clustering methods

Let us compare PQk-means and the traditional k-means using high-dimensional data

In [16]:
from sklearn.cluster import KMeans

In [17]:
X3 = numpy.random.random((1000, 1024))  # 1K 1024-dim vectors, for training 
X4 = numpy.random.random((10000, 1024)) # 10K 1024-dim vectors, for database
K = 100

In [18]:
# Train the encoder
encoder_large = pqkmeans.encoder.PQEncoder(num_subdim=4, Ks=256)
encoder_large.fit(X3)

# Encode the vectors to PQ-code
X4_pqcode = encoder_large.transform(X4)

Let's run the PQ-kmeans, and see the computational cost

In [19]:
%time clustered_pqkmeans = pqkmeans.clustering.PQKMeans(encoder=encoder_large, k=K).fit_predict(X4_pqcode)

CPU times: user 328 ms, sys: 0 ns, total: 328 ms
Wall time: 132 ms


Then, run the traditional k-means clustering 

In [20]:
%time clustered_kmeans = KMeans(n_clusters=K, n_jobs=-1).fit_predict(X4)

CPU times: user 1.95 s, sys: 40 ms, total: 1.99 s
Wall time: 50.7 s


PQk-means would be tens to hundreds of times faster than k-means depending on your machine. Then let's see the accuracy. Since the result of PQk-means is the approximation of that of k-means, k-means achieved the lower error:

In [21]:
_, pqkmeans_micro_average_error, _ = pqkmeans.evaluation.calc_error(clustered_pqkmeans, X4, K)
_, kmeans_micro_average_error, _ = pqkmeans.evaluation.calc_error(clustered_kmeans, X4, K)

print("PQk-means, micro avg error: ", pqkmeans_micro_average_error)
print("k-means, micro avg error: ", kmeans_micro_average_error)

PQk-means, micro avg error:  9.17477287633
k-means, micro avg error:  9.1501990608
